In [ ]:
# coding=utf-8
# 导入环境
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import paddle
from paddle.io import Dataset
from paddle.nn import Conv2D, MaxPool2D, Linear, Dropout, BatchNorm, AdaptiveAvgPool2D, AvgPool2D
import paddle.nn.functional as F
import paddle.nn as nn

##########################################################################################
##########################################################################################
# 图像分块、Embedding
# 定义了一个名为 PatchEmbed 的自定义图像块嵌入层。它将输入图像分成小的块，
# 并将每个块通过一个卷积层进行投影（Embedding），从而将图像转换为一个序列形式的张量。
##########################################################################################
# PatchEmbed 类继承自 paddle.nn.Layer，并实现了 forward 方法来定义前向传播操作。主要步骤如下：
# __init__ 方法：该方法初始化 PatchEmbed 类，接收以下参数：
# img_size：输入图像的大小，默认为 224，可以是一个整数或一个元组 [H, W]。
# patch_size：图像块的大小，默认为 16，同样可以是一个整数或一个元组 [H, W]。
# in_chans：输入图像的通道数，默认为 3（RGB图像）。
# embed_dim：块嵌入向量的维度，默认为 768。
##########################################################################################
# forward 方法：在前向传播过程中，通过卷积操作将图像块嵌入到一个向量序列中。具体步骤如下：
# 首先，输入图像的维度为 [B, C, H, W]，其中 B 是批次大小，C 是通道数，H 和 W 是图像的高度和宽度。
# 然后，代码通过一个卷积层 self.proj 将图像块投影（Embedding）到一个新的向量空间。
# 这个投影操作与图像块的大小和嵌入向量的维度有关。
# 接下来，通过 flatten 方法将卷积输出展平为一个形状为 [B, C, H*W] 的张量，其中 H*W 表示图像中的总块数。
# 最后，通过 transpose 方法将维度重新排列为 [B, H*W, C]，这样每个块就成为序列中的一个元素，
# 其中 H*W 表示序列的长度，C 是块嵌入向量的维度。
##########################################################################################
##########################################################################################
class PatchEmbed(nn.Layer):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        # 原始大小为int，转为tuple，即：img_size原始输入224，变换后为[224,224]
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        # 图像块的个数
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.proj = nn.Conv2D(
            in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        # in_chans：输入图像的通道数，默认为 3（RGB图像）。
        # embed_dim：块嵌入向量的维度，默认为 768。
        # kernel_size：卷积核大小，默认为 patch_size。
        # stride：卷积步长，默认为 patch_size。

    def forward(self, x):
        B, C, H, W = x.shape # 将x.shape的四个维度分别赋值给这四个变量，x是张量。
        assert H == self.img_size[0] and W == self.img_size[1], \
            "Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})." # 用于检查输入图像的大小是否与模型期望的大小相匹配。
        # [B, C, H, W] -> [B, C, H*W] ->[B, H*W, C]
        x = self.proj(x).flatten(2).transpose((0, 2, 1))
        # self.proj在__init__中定义为一个2D卷积核。
        # flatten(2)表示将第2维度的数据展平，即：[B, C, H*W]
        # transpose((0, 2, 1))表示将第1维度和第2维度的数据交换，即：[B, H*W, C]，这样刚好是序列数据的表示方式。
        return x